# Attention Toxicity Model

This notebook trains a model to detect toxicity in online comments. It uses a CNN architecture for text classification trained on the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973) and pre-trained GloVe embeddings which can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/).

This model is a modification of [example code](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) found in the [Keras Github repository](https://github.com/fchollet/keras) and released under an [MIT license](https://github.com/fchollet/keras/blob/master/LICENSE). For further details of this license, find it [online](https://github.com/fchollet/keras/blob/master/LICENSE) or in this repository in the file KERAS_LICENSE. 

## Usage Instructions
(TODO: nthain) - Move to README

Prior to running the notebook, you must:

* Download the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973)
* Download pre-trained [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip)
* (optional) To skip the training step, you will need to download a model and tokenizer file. We are looking into the appropriate means for distributing these (sometimes large) files.

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd

from model_tool import *

Using TensorFlow backend.


HELLO from model_tool


In [4]:
from model_tool import (DEFAULT_EMBEDDINGS_PATH,
DEFAULT_MODEL_DIR, DEFAULT_HPARAMS)
from keras.utils import to_categorical
from keras.layers import Multiply
from keras.callbacks import TensorBoard


#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))



class AttentionToxModel(ToxModel):
    def __init__(self,
                 model_name=None,
                 model_dir=DEFAULT_MODEL_DIR,
                 embeddings_path=DEFAULT_EMBEDDINGS_PATH,
                 hparams=None):
        self.model_dir = model_dir
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.hparams = DEFAULT_HPARAMS.copy()
        self.embeddings_path = embeddings_path
        if hparams:
            self.update_hparams(hparams)
        if model_name:
            self.load_model_from_name(model_name)
            self.load_probs_model(model_name)
        self.print_hparams()

    def load_probs_model(self, model_name):
        probs_model_name = model_name + "_probs"
        self.probs_model = load_model(
                os.path.join(
                    self.model_dir, '%s_model.h5' % probs_model_name))

    def save_prob_model(self):
        self.probs_model_name = self.model_name + "probs"

    def build_dense_attention_layer(self, input_tensor):
        # softmax
        attention_probs = Dense(self.hparams['max_sequence_length'],
                                activation='softmax',
                                name='attention_vec')(input_tensor)
        # context vector
        attention_mul = Multiply()([input_tensor, attention_probs])
        return {'attention_probs': attention_probs,
                'attention_preds': attention_mul}

    def train(
                self,
                training_data_path,
                validation_data_path, text_column,
                label_column, model_name):
        self.model_name = model_name
        self.save_hparams(model_name)

        train_data = pd.read_csv(training_data_path)
        valid_data = pd.read_csv(validation_data_path)

        print('Fitting tokenizer...')
        self.fit_and_save_tokenizer(train_data[text_column])
        print('Tokenizer fitted!')

        print('Preparing data...')
        train_text, train_labels = (self.prep_text(train_data[text_column]),
                                    to_categorical(train_data[label_column]))
        valid_text, valid_labels = (self.prep_text(valid_data[text_column]),
                                    to_categorical(valid_data[label_column]))
        print('Data prepared!')

        print('Loading embeddings...')
        self.load_embeddings()
        print('Embeddings loaded!')

        print('Building model graph...')
        self.build_model()
        print('Training model...')

        preds_save_path = os.path.join(
                            self.model_dir, '%s_model.h5' % self.model_name)
        probs_save_path = os.path.join(
                            self.model_dir, '%s_probs_model.h5'
                            % self.model_name)
        preds_callbacks = [ModelCheckpoint(
                            preds_save_path,
                            save_best_only=True,
                            verbose=self.hparams['verbose']), tensorboard]
        probs_callbacks = [ModelCheckpoint(
                            probs_save_path,
                            save_best_only=True,
                            verbose=self.hparams['verbose']),tensorboard]

        if self.hparams['stop_early']:
            early_stop = EarlyStopping(
                            min_delta=self.hparams['es_min_delta'],
                            monitor='val_loss',
                            patience=self.hparams['es_patience'],
                            verbose=self.hparams['verbose'], mode='auto')
            probs_callbacks.append(early_stop)
            preds_callbacks.append(early_stop)

        self.model.fit(train_text,
                       train_labels,
                       batch_size=self.hparams['batch_size'],
                       epochs=self.hparams['epochs'],
                       validation_data=(valid_text, valid_labels),
                       callbacks=preds_callbacks,
                       verbose=2)

        print('Model trained!')
        print('Best model saved to {}'.format(preds_save_path))
        print('Fitting probs model')

        self.probs_model.fit(
                    train_text,
                    train_labels,
                    batch_size=self.hparams['batch_size'],
                    epochs=self.hparams['epochs'],
                    validation_data=(valid_text, valid_labels),
                    callbacks=probs_callbacks,
                    verbose=2)
        self.probs_model = load_model(probs_save_path)
        print('Loading best model from checkpoint...')
        self.model = load_model(preds_save_path)
        print('Model loaded!')

    def build_model(self):
        print('print inside build model')
        sequence_input = Input(
                            shape=(self.hparams['max_sequence_length'],),
                            dtype='int32')
        embedding_layer = Embedding(
                            len(self.tokenizer.word_index) + 1,
                            self.hparams['embedding_dim'],
                            weights=[self.embedding_matrix],
                            input_length=self.hparams['max_sequence_length'],
                            trainable=self.hparams['embedding_trainable'])

        embedded_sequences = embedding_layer(sequence_input)
        x = embedded_sequences
        for filter_size, kernel_size, pool_size in zip(
                self.hparams['cnn_filter_sizes'],
                self.hparams['cnn_kernel_sizes'],
                self.hparams['cnn_pooling_sizes']):
            x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

        x = Flatten()(x)
        x = Dropout(self.hparams['dropout_rate'], name="Dropout")(x)
        x = Dense(250, activation='relu', name="Dense_RELU")(x)

        # build prediction model
        attention_dict = self.build_dense_attention_layer(x)
        preds = attention_dict['attention_preds']
        preds = Dense(2, name="preds_dense", activation='softmax')(preds)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.model = Model(sequence_input, preds)
        self.model.compile(
                loss='categorical_crossentropy',
                optimizer=rmsprop,
                metrics=['acc'])

        # now make probs model
        probs = attention_dict['attention_probs']
        probs = Dense(2, name='probs_dense')(probs)
        rmsprop = RMSprop(lr=self.hparams['learning_rate'])
        self.probs_model = Model(sequence_input, preds)
        self.probs_model.compile(
                loss='mse', optimizer=rmsprop, metrics=['acc'])
        # build probabilities model
        self.save_prob_model()


## Load Data

In [5]:
SPLITS = ['train', 'dev', 'test']

wiki = {}
debias = {}
random = {}
for split in SPLITS:
    wiki[split] = '../data/wiki_%s.csv' % split
    debias[split] = '../data/wiki_debias_%s.csv' % split
    random[split] = '../data/wiki_debias_random_%s.csv' % split

## Train Models

In [6]:
hparams = {'epochs': 20}

### Random model

In [ ]:
MODEL_NAME = 'cnn_debias_random_tox_v3'
debias_random_model = ToxModel(hparams=hparams)
debias_random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.18065, saving model to ../models/cnn_debias_random_tox_v3_model.h5
109s - loss: 0.2349 - acc: 0.9181 - val_loss: 0.1807 - val_acc: 0.9359
Epoch 2/20
Epoch 00001: val_loss improved from 0.18065 to 0.14876, saving model to ../models/cnn_debias_random_tox_v3_model.h5
108s - loss: 0.1624 - acc: 0.9407 - val_loss: 0.1488 - val_acc: 0.9461
Epoch 3/20
Epoch 00002: val_loss improved from 0.14876 to 

In [ ]:
MODEL_NAME = '20_atn_cnn_debias_random_tox_v3'
debias_random_model = AttentionToxModel(hparams=hparams)
debias_random_model.train(random['train'], random['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.23986, saving model to ../models/20_atn_cnn_debias_random_tox_v3_model.h5
105s - loss: 0.3014 - acc: 0.9066 - val_loss: 0.2399 - val_acc: 0.9078
Epoch 2/20
Epoch 00001: val_loss improved from 0.23986 to 0.20081, saving model to ../models/20_atn_cnn_debias_random_tox_v3_model.h5
107s - loss: 0.2208 - acc: 0.9125 - val_loss: 0.2008 - val_acc: 0.9265
Epoch 3/20
Epoch 00

In [ ]:
random_test = pd.read_csv(random['test'])
debias_random_model.score_auc(random_test['comment'], random_test['is_toxic'])

### Plain wikipedia model

In [7]:
MODEL_NAME = 'atn_cnn_wiki_tox_v3'
wiki_model = AttentionToxModel(hparams=hparams)
wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 4
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
print inside build model
Training model...
Train on 95692 samples, validate on 32128 samples
Epoch 1/4
Epoch 00000: val_loss improved from inf to 0.23601, saving model to ../models/atn_cnn_wiki_tox_v3_model.h5
100s - loss: 0.3048 - acc: 0.9020 - val_loss: 0.2360 - val_acc: 0.9045
Epoch 2/4
Epoch 00001: val_loss improved from 0.23601 to 0.19425, saving model to ../models/atn_cnn_wiki_tox_v3_model.h5
103s - loss: 0.2152 - acc: 0.9170 - val_loss: 0.1943 - val_acc: 0.9311
Epoch 3/4
Epoch 00002: val_loss improved from 

In [8]:
wiki_test = pd.read_csv(wiki['test'])
wiki_model.score_auc(wiki_test['comment'], wiki_test['is_toxic'])

0.93516059425530385

### Debiased model

In [10]:
MODEL_NAME = 'atn_cnn_debias_tox_v3'
debias_model = ToxModel(hparams=hparams)
debias_model.train(debias['train'], debias['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 4
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/4
Epoch 00000: val_loss improved from inf to 0.16953, saving model to ../models/atn_cnn_debias_tox_v3_model.h5
103s - loss: 0.2364 - acc: 0.9184 - val_loss: 0.1695 - val_acc: 0.9387
Epoch 2/4
Epoch 00001: val_loss improved from 0.16953 to 0.14930, saving model to ../models/atn_cnn_debias_tox_v3_model.h5
106s - loss: 0.1640 - acc: 0.9407 - val_loss: 0.1493 - val_acc: 0.9456
Epoch 3/4
Epoch 00002: val_loss improved from 0.14930 to 0.13845, s

In [11]:
debias_test = pd.read_csv(debias['test'])
debias_model.score_auc(debias_test['comment'], debias_test['is_toxic'])

0.95149490074750576